Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic

In [14]:
train_subset = 10000
beta = 0.01

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels], dtype=np.float32)
    )
    biases = tf.Variable(tf.zeros([num_labels], dtype=np.float32))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 801

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        
        if (step % 100 == 0):
            
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 46.756428
Training accuracy: 14.3%
Validation accuracy: 17.9%
Test accuracy: 18.1%
Loss at step 100: 12.071107
Training accuracy: 74.0%
Validation accuracy: 72.4%
Test accuracy: 80.0%
Loss at step 200: 4.563575
Training accuracy: 79.2%
Validation accuracy: 76.5%
Test accuracy: 84.6%
Loss at step 300: 2.006567
Training accuracy: 82.6%
Validation accuracy: 79.7%
Test accuracy: 87.3%
Loss at step 400: 1.137632
Training accuracy: 84.0%
Validation accuracy: 80.9%
Test accuracy: 88.2%
Loss at step 500: 0.839047
Training accuracy: 84.6%
Validation accuracy: 81.7%
Test accuracy: 88.5%
Loss at step 600: 0.735203
Training accuracy: 84.9%
Validation accuracy: 82.0%
Test accuracy: 88.7%
Loss at step 700: 0.698743
Training accuracy: 85.0%
Validation accuracy: 82.0%
Test accuracy: 88.7%
Loss at step 800: 0.685842
Training accuracy: 84.8%
Validation accuracy: 82.0%
Test accuracy: 88.7%


### Neural Network

In [13]:
batch_size = 128
image_size = 28
num_labels = 10
num_nodes = 1024
beta = 0.01

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(
        tf.float32,
        shape=(batch_size, image_size * image_size)
    )

    tf_train_labels = tf.placeholder(
        tf.float32,
        shape=(batch_size, num_labels)
    )
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    logits = tf.matmul(hidden, weights_2) + biases_2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * tf.nn.l2_loss(weights_1) + beta * tf.nn.l2_loss(weights_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2
    )
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction],
            feed_dict=feed_dict
        )
        
        if (step % 500 == 0):
            
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch Accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation Accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test Accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 3438.645508
Minibatch Accuracy: 18.0%
Validation Accuracy: 24.2%
Test Accuracy: 25.9%
Minibatch loss at step 500: 21.173054
Minibatch Accuracy: 85.9%
Validation Accuracy: 84.4%
Test Accuracy: 90.9%
Minibatch loss at step 1000: 0.950249
Minibatch Accuracy: 82.0%
Validation Accuracy: 82.9%
Test Accuracy: 90.0%
Minibatch loss at step 1500: 0.584362
Minibatch Accuracy: 85.9%
Validation Accuracy: 83.5%
Test Accuracy: 90.5%
Minibatch loss at step 2000: 0.603240
Minibatch Accuracy: 90.6%
Validation Accuracy: 83.3%
Test Accuracy: 90.6%
Minibatch loss at step 2500: 0.714579
Minibatch Accuracy: 84.4%
Validation Accuracy: 83.3%
Test Accuracy: 90.0%
Minibatch loss at step 3000: 0.762772
Minibatch Accuracy: 82.8%
Validation Accuracy: 83.4%
Test Accuracy: 90.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### Logistic

In [18]:
train_subset = 10000
beta = 0.01

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels], dtype=np.float32)
    )
    biases = tf.Variable(tf.zeros([num_labels], dtype=np.float32))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 20

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        
        _, l, predictions = session.run([optimizer, loss, train_prediction])
              
        print('Loss at step %d: %f' % (step, l))
        print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
        print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
        print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 49.277161
Training accuracy: 10.3%
Validation accuracy: 12.7%
Test accuracy: 14.0%
Loss at step 1: 45.554478
Training accuracy: 12.5%
Validation accuracy: 15.3%
Test accuracy: 16.4%
Loss at step 2: 42.693371
Training accuracy: 15.4%
Validation accuracy: 17.6%
Test accuracy: 19.1%
Loss at step 3: 40.486504
Training accuracy: 18.0%
Validation accuracy: 20.5%
Test accuracy: 22.0%
Loss at step 4: 38.783043
Training accuracy: 20.9%
Validation accuracy: 24.8%
Test accuracy: 27.4%
Loss at step 5: 37.459320
Training accuracy: 25.4%
Validation accuracy: 28.1%
Test accuracy: 31.3%
Loss at step 6: 36.364449
Training accuracy: 28.6%
Validation accuracy: 31.1%
Test accuracy: 34.5%
Loss at step 7: 35.418262
Training accuracy: 31.8%
Validation accuracy: 34.2%
Test accuracy: 37.7%
Loss at step 8: 34.585007
Training accuracy: 34.9%
Validation accuracy: 36.8%
Test accuracy: 40.4%
Loss at step 9: 33.838970
Training accuracy: 37.5%
Validation accuracy: 39.2%
Test accuracy: 43.0

### Neural Network

In [17]:
batch_size = 128
image_size = 28
num_labels = 10
num_nodes = 1024
beta = 0.01

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(
        tf.float32,
        shape=(batch_size, image_size * image_size)
    )

    tf_train_labels = tf.placeholder(
        tf.float32,
        shape=(batch_size, num_labels)
    )
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    logits = tf.matmul(hidden, weights_2) + biases_2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * tf.nn.l2_loss(weights_1) + beta * tf.nn.l2_loss(weights_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2
    )
    
num_steps = 20

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction],
            feed_dict=feed_dict
        )
           
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch Accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation Accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test Accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 3443.754639
Minibatch Accuracy: 8.6%
Validation Accuracy: 28.2%
Test Accuracy: 30.7%
Minibatch loss at step 1: 4379.582520
Minibatch Accuracy: 29.7%
Validation Accuracy: 27.4%
Test Accuracy: 29.4%
Minibatch loss at step 2: 4479.900879
Minibatch Accuracy: 25.0%
Validation Accuracy: 42.1%
Test Accuracy: 46.4%
Minibatch loss at step 3: 4059.132324
Minibatch Accuracy: 36.7%
Validation Accuracy: 28.9%
Test Accuracy: 31.6%
Minibatch loss at step 4: 3795.718994
Minibatch Accuracy: 22.7%
Validation Accuracy: 42.8%
Test Accuracy: 47.6%
Minibatch loss at step 5: 3552.430664
Minibatch Accuracy: 50.0%
Validation Accuracy: 57.8%
Test Accuracy: 64.0%
Minibatch loss at step 6: 3386.715820
Minibatch Accuracy: 59.4%
Validation Accuracy: 65.8%
Test Accuracy: 72.8%
Minibatch loss at step 7: 3192.325928
Minibatch Accuracy: 63.3%
Validation Accuracy: 67.3%
Test Accuracy: 74.2%
Minibatch loss at step 8: 3105.277344
Minibatch Accuracy: 63.3%
Validation Accuracy: 68.6%
Test Accuracy:

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [19]:
batch_size = 128
image_size = 28
num_labels = 10
num_nodes = 1024
beta = 0.01

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(
        tf.float32,
        shape=(batch_size, image_size * image_size)
    )

    tf_train_labels = tf.placeholder(
        tf.float32,
        shape=(batch_size, num_labels)
    )
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    drop = tf.nn.dropout(hidden, keep_prob=0.5)
    logits = tf.matmul(drop, weights_2) + biases_2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * tf.nn.l2_loss(weights_1) + beta * tf.nn.l2_loss(weights_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2
    )
    
num_steps = 20

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction],
            feed_dict=feed_dict
        )
           
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch Accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation Accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test Accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 3633.656250
Minibatch Accuracy: 9.4%
Validation Accuracy: 32.5%
Test Accuracy: 35.3%
Minibatch loss at step 1: 4109.480469
Minibatch Accuracy: 31.2%
Validation Accuracy: 24.2%
Test Accuracy: 25.6%
Minibatch loss at step 2: 4451.509766
Minibatch Accuracy: 24.2%
Validation Accuracy: 44.1%
Test Accuracy: 47.9%
Minibatch loss at step 3: 4031.727295
Minibatch Accuracy: 36.7%
Validation Accuracy: 41.5%
Test Accuracy: 44.4%
Minibatch loss at step 4: 3758.591309
Minibatch Accuracy: 39.1%
Validation Accuracy: 52.4%
Test Accuracy: 57.8%
Minibatch loss at step 5: 3389.154785
Minibatch Accuracy: 51.6%
Validation Accuracy: 59.9%
Test Accuracy: 66.9%
Minibatch loss at step 6: 3249.762695
Minibatch Accuracy: 53.1%
Validation Accuracy: 67.2%
Test Accuracy: 73.8%
Minibatch loss at step 7: 3107.837402
Minibatch Accuracy: 64.1%
Validation Accuracy: 66.1%
Test Accuracy: 72.6%
Minibatch loss at step 8: 3120.789551
Minibatch Accuracy: 67.2%
Validation Accuracy: 62.9%
Test Accuracy:

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [30]:
# https://stackoverflow.com/questions/36565430/adding-multiple-layers-to-tensorflow-causes-loss-function-to-become-nan

batch_size = 128
image_size = 28
num_labels = 10
num_nodes = 1024
beta = 0.01
num_steps = 3001

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(
        tf.float32,
        shape=(batch_size, image_size * image_size)
    )

    tf_train_labels = tf.placeholder(
        tf.float32,
        shape=(batch_size, num_labels)
    )
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    global_step = tf.Variable(0, trainable=False)
    initial_learning_rate = 0.01
    learning_rate = tf.train.exponential_decay(
        learning_rate=initial_learning_rate,
        global_step=global_step,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_nodes]))
    biases_2 = tf.Variable(tf.zeros([num_nodes]))
    
    weights_3 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_3 = tf.Variable(tf.zeros([num_labels]))
    
    hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    drop_1 = tf.nn.dropout(hidden_1, keep_prob=0.5)
    hidden_2 = tf.nn.relu(tf.matmul(drop_1, weights_2) + biases_2)
    drop_2 = tf.nn.dropout(hidden_2, keep_prob=0.5)
    logits = tf.matmul(drop_2, weights_3) + biases_3
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * tf.nn.l2_loss(weights_1) + beta * tf.nn.l2_loss(weights_2) + beta * tf.nn.l2_loss(weights_3)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2), weights_3) + biases_3
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2), weights_3) + biases_3
    )

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction],
            feed_dict=feed_dict
        )
           
        if (step % 500 == 0):
   
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch Accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation Accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test Accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 21125.673828
Minibatch Accuracy: 11.7%
Validation Accuracy: 21.7%
Test Accuracy: 22.6%
Minibatch loss at step 500: 6756.416504
Minibatch Accuracy: 71.9%
Validation Accuracy: 81.4%
Test Accuracy: 88.4%
Minibatch loss at step 1000: 6108.448730
Minibatch Accuracy: 72.7%
Validation Accuracy: 80.9%
Test Accuracy: 88.0%
Minibatch loss at step 1500: 5687.277832
Minibatch Accuracy: 77.3%
Validation Accuracy: 79.8%
Test Accuracy: 87.4%
Minibatch loss at step 2000: 5395.458984
Minibatch Accuracy: 78.1%
Validation Accuracy: 79.8%
Test Accuracy: 86.8%
Minibatch loss at step 2500: 5176.371094
Minibatch Accuracy: 69.5%
Validation Accuracy: 79.7%
Test Accuracy: 87.2%
Minibatch loss at step 3000: 5000.072754
Minibatch Accuracy: 68.8%
Validation Accuracy: 78.8%
Test Accuracy: 86.2%
